# First load the model and output the performance of each model on val and test set

In [1]:
pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=c5cb02a11952181426a7f88077ec8da0f525e0ad04d8410e7c1e2bdab861b497
  Stored in directory: /global/u2/t/tianle/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install datetime

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 52 kB 1.2 MB/s  eta 0:00:01
     |████████████████████████████████| 255 kB 20.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import xarray as xr
import h5py

from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.layers import Input, LeakyReLU, Dense, Activation, Flatten, Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, Reshape, Concatenate
from tensorflow.keras.models import Model

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [5]:
with h5py.File('processed_data_ae/np_data.h5', 'r') as hf:
    data = hf['np_data'][:]

print(data.shape)

(8530, 192, 192, 2)


In [6]:
print(np.max(data), np.min(data))

19.913855 -16.306587


In [7]:
#First split data into train+validation and test set
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

#Next split training again into train and validation
X_train, X_val = train_test_split(X_train, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(np.max(X_train), np.max(X_val), np.max(X_test), np.min(X_train), np.min(X_val), np.min(X_test))

(5118, 192, 192, 2)
(1706, 192, 192, 2)
(1706, 192, 192, 2)
19.913855 18.198473 19.736868 -16.306587 -16.134562 -16.26034


In [8]:
def encoder(latent_dim):
    '''
    return an encoder which encodes the input image into a latent vector with dimension latent_dim
    '''
    
    X_input = Input((192, 192, 2))
    
    #FIXME Should we add BN layer? I currently add that between conv and relu for the first 4 sets of layers
    X = Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding="same")(X_input)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    #FIXME Should we add some dropout layer to regularize the model? 
    #I didn't do that, but need to look at train/val error
    
    X = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Flatten()(X)
    X = Dense(units=latent_dim)(X)
    #FIXME Should we add an activation layer here? I didn't do it
    
    model = Model(inputs = X_input, outputs = X)
    return model

In [9]:
def decoder(latent_dim):
    '''
    return an encoder which encodes the input image into a latent vector with dimension latent_dim
    '''
    
    X_input = Input((latent_dim))
    
    X = Dense(units=3*3*64, input_dim=latent_dim)(X_input)
    X = Reshape((3,3,64))(X)
    
    X = Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=32, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=32, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=16, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=16, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(filters=2, kernel_size=(3,3), strides=(1,1), padding="same")(X)    
    
    model = Model(inputs = X_input, outputs = X)
    return model

In [10]:
class Autoencoder(Model):
    def __init__(self, encoder, decoder):
        super(Autoencoder, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [16]:
encoder_4 = encoder(4)
decoder_4 = decoder(4)
autoencoder_4 = Autoencoder(encoder_4, decoder_4)
autoencoder_4.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_4.load_weights("autoencoder_4/ckp/")
print("val loss: ", autoencoder_4.evaluate(X_val, X_val))
print("test loss: ", autoencoder_4.evaluate(X_test, X_test))

54/54 [==============================] - 1s 7ms/step - loss: 0.3671
val loss:  0.3671454191207886
54/54 [==============================] - 0s 7ms/step - loss: 0.3659
test loss:  0.36588191986083984


In [14]:
encoder_18 = encoder(18)
decoder_18 = decoder(18)
autoencoder_18 = Autoencoder(encoder_18, decoder_18)
autoencoder_18.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_18.load_weights("autoencoder_18/ckp/")
print("val loss: ", autoencoder_18.evaluate(X_val, X_val))
print("test loss: ", autoencoder_18.evaluate(X_test, X_test))

54/54 [==============================] - 0s 7ms/step - loss: 0.2130
val loss:  0.2129608690738678
54/54 [==============================] - 0s 6ms/step - loss: 0.2150
test loss:  0.21500757336616516


In [17]:
encoder_36 = encoder(36)
decoder_36 = decoder(36)
autoencoder_36 = Autoencoder(encoder_36, decoder_36)
autoencoder_36.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_36.load_weights("autoencoder_36/ckp/")
print("val loss: ", autoencoder_36.evaluate(X_val, X_val))
print("test loss: ", autoencoder_36.evaluate(X_test, X_test))

54/54 [==============================] - 1s 7ms/step - loss: 0.1667
val loss:  0.16668541729450226
54/54 [==============================] - 0s 7ms/step - loss: 0.1688
test loss:  0.16877079010009766


In [18]:
encoder_72 = encoder(72)
decoder_72 = decoder(72)
autoencoder_72 = Autoencoder(encoder_72, decoder_72)
autoencoder_72.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_72.load_weights("autoencoder_72/ckp/")
print("val loss: ", autoencoder_72.evaluate(X_val, X_val))
print("test loss: ", autoencoder_72.evaluate(X_test, X_test))

54/54 [==============================] - 1s 7ms/step - loss: 0.1427
val loss:  0.14272011816501617
54/54 [==============================] - 0s 6ms/step - loss: 0.1447
test loss:  0.14470382034778595


In [19]:
encoder_144 = encoder(144)
decoder_144 = decoder(144)
autoencoder_144 = Autoencoder(encoder_144, decoder_144)
autoencoder_144.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_144.load_weights("autoencoder_144/ckp/")
print("val loss: ", autoencoder_144.evaluate(X_val, X_val))
print("test loss: ", autoencoder_144.evaluate(X_test, X_test))

54/54 [==============================] - 1s 7ms/step - loss: 0.1120
val loss:  0.11195648461580276
54/54 [==============================] - 0s 6ms/step - loss: 0.1144
test loss:  0.11435379087924957


In [20]:
class Autoencoder_hier_2(Model):
    def __init__(self, encoder, decoder, ae):
        super(Autoencoder_hier_2, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder
        self.ae = ae
        self.ae.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        latent_1 = self.ae.encoder(x)
        latent_all = Concatenate(axis=-1)([encoded, latent_1])
        decoded = self.decoder(latent_all)
        return decoded

In [21]:
encoder_18_sub2 = encoder(18)
decoder_18_sub2 = decoder(36)
autoencoder_18_hier_2 = Autoencoder_hier_2(encoder_18_sub2, decoder_18_sub2, autoencoder_18)
autoencoder_18_hier_2.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_18_hier_2.load_weights("autoencoder_18_hier_2/ckp/")
print("val loss: ", autoencoder_18_hier_2.evaluate(X_val, X_val))
print("test loss: ", autoencoder_18_hier_2.evaluate(X_test, X_test))

54/54 [==============================] - 1s 8ms/step - loss: 0.1828
val loss:  0.18277886509895325
54/54 [==============================] - 0s 8ms/step - loss: 0.1847
test loss:  0.18471655249595642


In [22]:
class Autoencoder_hier_3(Model):
    def __init__(self, encoder, decoder, ae):
        super(Autoencoder_hier_3, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder
        self.ae = ae
        self.ae.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        latent_1 = self.ae.ae.encoder(x)
        latent_2 = self.ae.encoder(x)
        latent_all = Concatenate(axis=-1)([encoded, latent_1, latent_2])
        decoded = self.decoder(latent_all)
        return decoded

In [23]:
encoder_18_sub3 = encoder(18)
decoder_18_sub3 = decoder(54)
autoencoder_18_hier_3 = Autoencoder_hier_3(encoder_18_sub3, decoder_18_sub3, autoencoder_18_hier_2)
autoencoder_18_hier_3.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_18_hier_3.load_weights("autoencoder_18_hier_3/ckp/")
print("val loss: ", autoencoder_18_hier_3.evaluate(X_val, X_val))
print("test loss: ", autoencoder_18_hier_3.evaluate(X_test, X_test))

54/54 [==============================] - 1s 10ms/step - loss: 0.1666
val loss:  0.16658547520637512
54/54 [==============================] - 1s 9ms/step - loss: 0.1686
test loss:  0.16864728927612305


In [24]:
class Autoencoder_hier_4(Model):
    def __init__(self, encoder, decoder, ae):
        super(Autoencoder_hier_4, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder
        self.ae = ae
        self.ae.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        latent_1 = self.ae.ae.ae.encoder(x)
        latent_2 = self.ae.ae.encoder(x)
        latent_3 = self.ae.encoder(x)
        latent_all = Concatenate(axis=-1)([encoded, latent_1, latent_2, latent_3])
        decoded = self.decoder(latent_all)
        return decoded

In [25]:
encoder_18_sub4 = encoder(18)
decoder_18_sub4 = decoder(72)
autoencoder_18_hier_4 = Autoencoder_hier_4(encoder_18_sub4, decoder_18_sub4, autoencoder_18_hier_3)
autoencoder_18_hier_4.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_18_hier_4.load_weights("autoencoder_18_hier_4/ckp/")
print("val loss: ", autoencoder_18_hier_4.evaluate(X_val, X_val))
print("test loss: ", autoencoder_18_hier_4.evaluate(X_test, X_test))

54/54 [==============================] - 1s 11ms/step - loss: 0.1572
val loss:  0.15718421339988708
54/54 [==============================] - 1s 11ms/step - loss: 0.1589
test loss:  0.15886655449867249


# Look at the performance of PCA

In [27]:
from sklearn.decomposition import PCA

In [46]:
def pca_util(data_train, data_val, data_test, n_components):
    N_train = data_train.shape[0]
    N_val   = data_val.shape[0]
    N_test  = data_test.shape[0]
    
    data_train_in = data_train.reshape(N_train,-1)
    data_val_in   = data_val.reshape(N_val,-1)
    data_test_in  = data_test.reshape(N_test,-1)
    
    pca = PCA(n_components=n_components)
    pca.fit(data_train_in)
    
    data_train_pca = pca.transform(data_train_in)
    data_val_pca   = pca.transform(data_val_in)
    data_test_pca  = pca.transform(data_test_in)
    
    data_train_out = pca.inverse_transform(data_train_pca)
    data_val_out   = pca.inverse_transform(data_val_pca)
    data_test_out  = pca.inverse_transform(data_test_pca)
    
    mse = tf.keras.losses.MeanSquaredError()
    
    train_loss = mse(data_train_in, data_train_out).numpy()
    val_loss   = mse(data_val_in, data_val_out).numpy()
    test_loss  = mse(data_test_in, data_test_out).numpy()
    
    return pca, train_loss, val_loss, test_loss

In [49]:
def pca_all_together(n_components):
    pca, train_loss, val_loss, test_loss = pca_util(X_train, X_val, X_test, n_components)
    print("Latent_dim = {}, train_loss = {}, val_loss = {}, test_loss = {}".format(n_components, train_loss, val_loss, test_loss))

In [51]:
pca_all_together(4)
pca_all_together(18)
pca_all_together(36)
pca_all_together(72)
pca_all_together(144)
pca_all_together(144*16)

Latent_dim = 4, train_loss = 0.6355684995651245, val_loss = 0.6420252919197083, test_loss = 0.6480738520622253
Latent_dim = 18, train_loss = 0.28684109449386597, val_loss = 0.2909291386604309, test_loss = 0.29305580258369446
Latent_dim = 36, train_loss = 0.18933157622814178, val_loss = 0.19471284747123718, test_loss = 0.19567106664180756
Latent_dim = 72, train_loss = 0.11902341991662979, val_loss = 0.1252274513244629, test_loss = 0.1261128932237625
Latent_dim = 144, train_loss = 0.0722622275352478, val_loss = 0.07911373674869537, test_loss = 0.07992440462112427
Latent_dim = 2304, train_loss = 0.005508307367563248, val_loss = 0.019411679357290268, test_loss = 0.019859718158841133
